In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import matplotlib.pyplot as plt
import seaborn as sns
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
rain_fall_df=pd.read_csv('/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv')

In [ ]:
rain_fall_df.RainTomorrow.value_counts()

In [ ]:
rain_fall_df.info()

### Check missing values

Check missing values percentages

In [ ]:
for col in rain_fall_df.columns:
    print(round((1-rain_fall_df[col].count()/len(rain_fall_df.index)),2)*100,col)

In [ ]:
rain_fall_df.Sunshine=rain_fall_df.Sunshine.fillna(rain_fall_df.Sunshine.mean())
rain_fall_df.Cloud9am=rain_fall_df.Cloud9am.fillna(rain_fall_df.Cloud9am.mean())
rain_fall_df.Cloud3pm=rain_fall_df.Cloud3pm.fillna(rain_fall_df.Cloud3pm.mean())
rain_fall_df.Evaporation=rain_fall_df.Evaporation.fillna(rain_fall_df.Evaporation.mean())

In [ ]:
rain_fall_df

#### fill missing values for other variables

In [ ]:
rain_fall_df.info()

In [ ]:
for col in rain_fall_df.columns:
    if str(rain_fall_df[col].dtype)=="object":
        print("yes")
        rain_fall_df[col]=rain_fall_df[col].fillna(rain_fall_df[col].mode()[0])
    else:
        rain_fall_df[col]=rain_fall_df[col].fillna(rain_fall_df[col].mean())
        

In [ ]:
rain_fall_df.info()

### Check for outliers

Looks like there are outliers in below varaibles
- Rainfall
- WindGustSpeed
- Windspeed9am
- WindSpeed3pm


In [ ]:
plt.figure(figsize=(10,50))
sns.boxplot(data=rain_fall_df,x='RainTomorrow',y='Rainfall')
plt.show()

In [ ]:
rain_fall_df[rain_fall_df['Rainfall']>150]

In [ ]:
rain_fall_df=rain_fall_df[rain_fall_df['Rainfall']<150]

In [ ]:
plt.figure(figsize=(10,50))
sns.boxplot(data=rain_fall_df,x='RainTomorrow',y='Rainfall')
plt.show()

In [ ]:
#### outliers WindGustSpeed


In [ ]:
sns.boxplot(data=rain_fall_df,x='RainTomorrow',y='WindGustSpeed')

#### Windspeed9am

In [ ]:
sns.boxplot(data=rain_fall_df,x='RainTomorrow',y='WindSpeed9am')

In [ ]:
rain_fall_df=rain_fall_df[rain_fall_df['WindSpeed9am']<70]

In [ ]:
rain_fall_df

#### WindSpeed3pm

In [ ]:
sns.boxplot(data=rain_fall_df,x='RainTomorrow',y='WindSpeed3pm')

In [ ]:
rain_fall_df=rain_fall_df[rain_fall_df['WindSpeed3pm']<70]

In [ ]:
rain_fall_df

#### Convert RainToday,RainTomorrow categorical into numerical


In [ ]:
rain_fall_df.RainToday=rain_fall_df.RainToday.apply(lambda x: 1 if x=='Yes' else 0)

In [ ]:
rain_fall_df.RainTomorrow=rain_fall_df.RainTomorrow.apply(lambda x: 1 if x.strip()=='Yes' else 0)

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'Country'. 
rain_fall_df['Location']= label_encoder.fit_transform(rain_fall_df['Location']) 
rain_fall_df['WindGustDir']= label_encoder.fit_transform(rain_fall_df['WindGustDir']) 
rain_fall_df['WindDir9am']= label_encoder.fit_transform(rain_fall_df['WindDir9am']) 
rain_fall_df['WindDir3pm']= label_encoder.fit_transform(rain_fall_df['WindDir3pm']) 
print(rain_fall_df)

In [ ]:
rain_fall_df['year']=pd.DatetimeIndex(rain_fall_df['Date']).year
rain_fall_df['month']=pd.DatetimeIndex(rain_fall_df['Date']).month
rain_fall_df['day']=pd.DatetimeIndex(rain_fall_df['Date']).day

In [ ]:
rain_fall_df

In [ ]:
rain_fall_df.drop('Date',axis=1,inplace=True)

In [ ]:
rain_fall_df['RainToday'].value_counts()

In [ ]:
rain_fall_df

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(rain_fall_df.corr(),annot=True)
plt.show()

There is high correlation between
 - Pressure9am and Presure3pm
 - Temp9am and temp3pm
 - Mintemp and temp3pm
 - Maxtemp and temp3pm
 
So i am going to drop below columns
  Pressure9am,Temp9am,Mintemp,Maxtemp

In [ ]:
rain_fall_df=rain_fall_df.drop(['Pressure9am','Temp9am','MinTemp','MaxTemp'],axis=1)

In [ ]:
rain_fall_df

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [ ]:
Y=rain_fall_df.pop('RainTomorrow')
X=rain_fall_df

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

In [ ]:
X_train

In [ ]:
model=XGBClassifier(random_state=42,verbosity = 0,n_jobs=-1)
model.fit(X_train, Y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(accuracy_score(y_pred,Y_test))